In [ ]:
# Dependencies

from bs4 import BeautifulSoup as bs
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import requests
import pymongo
import time

In [ ]:
# Set up splinter browser

executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

time.sleep(1)

In [ ]:
# First mars website to scrape

url1 = 'https://redplanetscience.com/'

browser.visit(url1)

time.sleep(1)

In [ ]:
# Soupify

html = browser.html
soup = bs(html, 'html.parser')

In [ ]:
news_title = soup.find("div", class_="content_title").text

In [ ]:
news_paragraph = soup.find("div", class_="article_teaser_body").text

In [ ]:
# Deliverables: news_title, news_paragraph

In [ ]:
# Second mars website to scrape

url2 = 'https://spaceimages-mars.com/'

browser.visit(url2)

time.sleep(1)

In [ ]:
# Soupify

html = browser.html
soup = bs(html, 'html.parser')

In [ ]:
browser.links.find_by_partial_text('FULL IMAGE').click()

In [ ]:
# Soupify 

html = browser.html
soup = bs(html, 'html.parser')

In [ ]:
image_rel_url = soup.find("img", class_="fancybox-image")["src"]

In [ ]:
image_featured_url = f"{url2}{image_rel_url}"

In [ ]:
# Deliverable: image_featured_url

In [ ]:
# Third mars website to scrape

url3 = "https://galaxyfacts-mars.com/"

In [ ]:
# Scrape the html tables

tables = pd.read_html(url3, index_col=None, encoding="utf-8")

In [ ]:
# Choose the one that has mars facts

tables[1]

In [ ]:
# Render the table as a dataframe

df = tables[1]

In [ ]:
# Do cleaning in the dataframe with pandas

df = df.rename(columns={0:"Characteristic", 1:"Data"})

In [ ]:
df["Characteristic"] = df["Characteristic"].str.rstrip(":")

In [ ]:
# Render back to an html table

html_table = df.to_html(classes="table table-striped", index=False)

In [ ]:
# Deliverable: html_table

In [ ]:
# Fourth mars website to scrape

url4 = 'https://marshemispheres.com/'

browser.visit(url4)

time.sleep(1)

In [ ]:
# Soupify

html = browser.html
soup = bs(html, 'html.parser')

In [ ]:
# Iterate over image_tags and collect tif_titles

image_tags = soup.find_all("img", class_="thumb")

img_titles = []

for tag in image_tags:
    
    img_titles.append(tag["alt"].replace(" thumbnail", ""))

In [ ]:
# Iterate over links_found and collect jpg_img_urls

links_found = browser.find_by_css('h3').links.find_by_partial_text('Hemisphere Enhanced')

jpg_img_urls = []

for link in links_found:
    
    img_link = link["href"]    
        
    browser.visit(img_link)

    time.sleep(2)   

    html = browser.html
    soup = bs(html, 'html.parser')
    
    img_tag = soup.find("img", class_="wide-image")    
    
    img_url = f'{url4}{img_tag["src"]}'         
    
    jpg_img_urls.append(img_url)
    
    browser.back()
    
    time.sleep(2)

In [ ]:
# Zip titles and urls into a list

photo_data = list(zip(img_titles, jpg_img_urls))

In [ ]:
# Deliverables: img_titles, jpg_img_urls, photo_data

In [ ]:
# Write all scraped artefacts to dictionary

mars_dict = {
    "news_title" : news_title,
    "news_paragraph" : news_paragraph,
    "image_featured_url" : image_featured_url,
    "html_table" : html_table,
    "photo_data" : photo_data     
}

In [ ]:
# Quit the browser

browser.quit()